In [9]:
import pandas as pd
import os
from datetime import datetime as dt
base = os.getcwd() + "\\"

In [15]:
matches_raw =      pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'League Matches')
points_table_raw = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Points Table')
mapping      = pd.read_excel(base + "IPL_2024.xlsx", sheet_name = 'Mapping')

display(mapping)

,Team Full Name,Team Half Name,Team Abbr
0,Chennai Super Kings,Super Kings,CSK
1,Punjab Kings,Kings,PBKS
2,Kolkata Knight Riders,Knight Riders,KKR
3,Rajasthan Royals,Royals,RR
4,Gujarat Titans,Titans,GT
5,Royal Challengers Bengaluru,Royal Challengers,RCB
6,Sunrisers Hyderabad,Sunrisers,SRH
7,Lucknow Super Giants,Super Giants,LSG
8,Delhi Capitals,Capitals,DC
9,Mumbai Indians,Indians,MI


In [16]:
matches = matches_raw.copy()
matches['DATETIME'] = pd.to_datetime(matches['DATE'] + " " + matches['TIME'])

matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'HOME', 'Team Abbr':'HOME_TEAM'}),on = 'HOME', how = 'left')
matches = matches.merge(mapping[['Team Full Name','Team Abbr']].rename(columns = {'Team Full Name':'AWAY', 'Team Abbr':'AWAY_TEAM'}),on = 'AWAY', how = 'left')

print(dt.now())
curr_date = dt(2024,5,5)

future_matches = matches[matches['DATETIME']>curr_date]
display(future_matches)

2024-05-05 13:11:47.215914


,MATCH\nNO,MATCH\nDAY,DATE,DAY,TIME,HOME,AWAY,STADIUM,DATETIME,HOME_TEAM,AWAY_TEAM
52,53,45,05-May-24,Sun,3:30PM,Punjab Kings,Chennai Super Kings,Dharamshala,2024-05-05 15:30:00,PBKS,CSK
53,54,45,05-May-24,Sun,7:30PM,Lucknow Super Giants,Kolkata Knight Riders,Lucknow,2024-05-05 19:30:00,LSG,KKR
54,55,46,06-May-24,Mon,7:30PM,Mumbai Indians,Sunrisers Hyderabad,Mumbai,2024-05-06 19:30:00,MI,SRH
55,56,47,07-May-24,Tue,7:30PM,Delhi Capitals,Rajasthan Royals,Delhi,2024-05-07 19:30:00,DC,RR
56,57,48,08-May-24,Wed,7:30PM,Sunrisers Hyderabad,Lucknow Super Giants,Hyderabad,2024-05-08 19:30:00,SRH,LSG
57,58,49,09-May-24,Thu,7:30PM,Punjab Kings,Royal Challengers Bengaluru,Dharamshala,2024-05-09 19:30:00,PBKS,RCB
58,59,50,10-May-24,Fri,7:30PM,Gujarat Titans,Chennai Super Kings,Ahmedabad,2024-05-10 19:30:00,GT,CSK
59,60,51,11-May-24,Sat,7:30PM,Kolkata Knight Riders,Mumbai Indians,Kolkata,2024-05-11 19:30:00,KKR,MI
60,61,52,12-May-24,Sun,3:30PM,Chennai Super Kings,Rajasthan Royals,Chennai,2024-05-12 15:30:00,CSK,RR
61,62,52,12-May-24,Sun,7:30PM,Royal Challengers Bengaluru,Delhi Capitals,Bengaluru,2024-05-12 19:30:00,RCB,DC


In [19]:
points_table = points_table_raw.copy()
points_table = points_table.merge(mapping[['Team Half Name','Team Abbr']].rename(columns = {'Team Half Name':'TEAM', 'Team Abbr':'TEAM_NAME'}),on = 'TEAM', how = 'left')
display(points_table)

,TEAM,MATCHES,WON,LOST,PTS,NRR,TEAM_NAME_x,TEAM_NAME_y
0,Royals,10,8,2,16,0.622,RR,RR
1,Knight Riders,10,7,3,14,1.098,KKR,KKR
2,Super Giants,10,6,4,12,0.094,LSG,LSG
3,Sunrisers,10,6,4,12,0.072,SRH,SRH
4,Super Kings,10,5,5,10,0.627,CSK,CSK
5,Capitals,11,5,6,10,-0.442,DC,DC
6,Royal Challengers,11,4,7,8,-0.049,RCB,RCB
7,Kings,10,4,6,8,-0.062,PBKS,PBKS
8,Titans,11,4,7,8,-1.320,GT,GT
9,Indians,11,3,8,6,-0.356,MI,MI
